###### 參考資料
1. [[實戰系列] 使用 Keras 搭建一個 GAN 魔法陣（模型）](https://ithelp.ithome.com.tw/articles/10208478)
2. [[魔法陣系列] Generative Adversarial Network（GAN）之術式解析](https://ithelp.ithome.com.tw/articles/10207229)
3. [[魔法陣系列] AutoEncoder 之術式解析](https://ithelp.ithome.com.tw/articles/10206869)

4. [[魔法陣系列] AutoEncoder 之應用場景](https://ithelp.ithome.com.tw/articles/10207903)

In [55]:
import numpy as np
import os, cv2

In [56]:
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, multiply, GaussianNoise
from keras.layers import BatchNormalization, Activation, Embedding, ZeroPadding2D
from keras.layers import MaxPooling2D, merge
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam

E:\Anaconda3\envs\TF-Ke\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [57]:
import keras.backend as K

###### data load

In [58]:
dataNPYname = "data.npy"
dataDictionary = np.load(dataNPYname).item()
print(dataDictionary.keys())

dict_keys(['x', 'y'])


In [59]:
dataAmount = len(dataDictionary["y"])
idx_arr = np.arange(dataAmount)
np.random.shuffle(idx_arr);

## 自備函數

##### 函數 - 測試用

In [60]:
def CheckType(intVar):
    print("max:",   intVar.max())
    print("min:",   intVar.min())
    print("shape:", intVar.shape)
    print("type:",  intVar.dtype)
    return

##### 函數 - 輸出用

In [61]:
def ResultImgShow(inputImg, modelName, strImgLabel = "TMP"):
    # 輸出參數設置
    img_amount = inputImg.shape[0]
    cols_output = 50
    rows_output = img_amount // cols_output
#     # 改變大小用，但這裡有確切大小所以不必處理
#     if rows_output < 3:
#         cols_output = 10
#         rows_output = img_amount // cols_output
#         if rows_output < 3:
#             cols_output = 5
#             rows_output = img_amount // cols_output
    if rows_output > cols_output:
        rows_output = cols_output
    # 輸出設置 - 水平堆疊再垂直堆疊
    ## v - 初始
    img_h = inputImg[0]
    for i in range(1, cols_output):
        img_tmp = inputImg[i]
        img_h = np.hstack((img_h, img_tmp))
    img_v = img_h.copy()
    ## v - Loop
    for j in range(1, rows_output):
        ## h - 初始
        img_h = inputImg[j * cols_output]
        ## h - Loop
        for i in range(1, cols_output):
            img_tmp = inputImg[j * cols_output + i]
            img_h = np.hstack((img_h, img_tmp))
        img_v = np.vstack((img_v, img_h))
    # 顯示 與 存取
    imwrite(modelName.split(".")[0] +"_"+ strImgLabel+ ".png", img_v)
    plt.imshow(img_v, cmap = "gray")
#     plt.savefig(model_name.split(".")[0] +"_"+ strImgLabel+ ".png")
    plt.show()
    plt.close()
    return##### 函數 - 輸出用

###### 函數 - 計算誤差

In [62]:
def PSNR_ALL(testData, truthData, strImgLabel = "TMP", boolTest = False):
    """ """
#     dataAmount = 1 if testData.shape < 3 else testData.shape[0]
#     if testData.shape < 3:
#         dataAmount = 1
#     else:
    dataAmount = testData.shape[0]
    psnrSum_test = 0
    for i in range(dataAmount):
        groundTruth   = truthData[i, :,:]
        predictResult = testData[i, :,:]
        psnrTmp = float(compare_psnr(groundTruth, predictResult))
        psnrSum_test += psnrTmp
    psnrSum_test /= dataAmount
    print(strImgLabel, "PSNR:", psnrSum_test)
    return psnrSum_test

In [63]:
def SSIM_ALL(testData, truthData, strImgLabel = "TMP", boolTest = False):
    dataAmount = testData.shape[0]
    ssimSum_test = 0
    for i in range(dataAmount):
        groundTruth   = truthData[i, :,:]
        predictResult = testData[i, :,:]
        ssimTmp = float(compare_ssim(groundTruth, predictResult))
        ssimSum_test += ssimTmp
    ssimSum_test /= dataAmount
    print(strImgLabel, "SSIM:", ssimSum_test)
    return ssimSum_test

###### 函數 - 預處理與後處理

In [64]:
def DataPreProcess(inputData, img_rows = 28, img_cols = 28):
    outputData = inputData.reshape(inputData.shape[0], img_rows * img_cols)
    outputData = outputData.astype('float32') /255 -0.5
    outputData *= 2
    return outputData

In [65]:
def DataRecovery(inputData, img_rows = 28, img_cols = 28):
    outputData = ((inputData /2)+0.5) * 255
    outputData = outputData.reshape((outputData.shape[0], img_rows, img_cols))
    outputData = outputData.astype('uint8')
    return outputData

# 主程式

In [ ]:
class GAN_AAE:
    def __init__(self, inputDict):
        # LOAD DATA，define rows, cols
        self._x, self._y = self.LoadData(inputDict)
        self.latent_dim = 2
        
        optimizer = Adam(0.0002, 0.5)
        
        
        return
    
    def LoadData(self, inputDict):
        x = inputDict["x"]
        y = inputDict["y"]
        # define shape
        tmpImg = inputDict["x"][0]
        self._img_rows, self._img_cols = tmpImg.shape[:2]
        if len(tmpImg.shape)<=2 :
            self._img_channels = 1
        else:
            self._img_channels = tmpImg.shape[2]
        self._img_shape = (self._img_rows, self._img_cols, self._img_channels)
        del tmpImg
        
        return x, y
    
#     def Build